In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
import numpy as np
import argparse
from tqdm.notebook import tqdm
from PIL import Image
from pathlib import Path
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import time
import clip

c:\Users\Shanette\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
VIDEO_FOLDER_ID = "10FI2eMd04bg7xXJlYDQvlQE2bwQB3DAV"
OUTPUT_FOLDER_ID = "15eifVkFWOhUxgOlc4Uf_i-WorhlagTKF"

FRAME_SAMPLE_RATE = 30
BATCH_SIZE = 32

# Where to temporarily download videos. "." means the same folder as the notebook.
TEMP_FOLDER = Path(".")
os.chdir("..")

In [3]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
print("Google Drive authentication successful.")

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=301401544903-fbfbldqjv6puuk60klvg7fdjefmj93vc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.
Google Drive authentication successful.


In [4]:
print("Setting up model and device...")

MODEL_NAME = "ViT-B/32"

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(MODEL_NAME, device=device)
model.eval()
print(f"Using device: {device}")

Setting up model and device...
Using device: cpu


In [5]:
def extract_clip_embeddings(
    video_path: Path, 
    model, 
    preprocess, 
    device: str, 
    frame_sample_rate: int = 30, 
    batch_size: int = 32
) -> np.ndarray:
    """Extracts mean-pooled CLIP visual embeddings from a single video file."""
    if not video_path.exists():
        raise FileNotFoundError(f"Video file not found: {video_path}")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {video_op_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    all_features = []
    frame_batch = []
    frame_idx = 0

    pbar = tqdm(total=frame_count, desc=f"Frames for {video_path.name}", leave=False)

    with torch.no_grad():
        while True:
            ret, frame = cap.read()
            if not ret: break
            pbar.update(1)

            if frame_idx % frame_sample_rate == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(frame_rgb)
                frame_batch.append(pil_img)

                if len(frame_batch) == batch_size:
                    image_inputs = torch.stack(
                        [preprocess(img) for img in frame_batch]
                    ).to(device)
                    image_features = model.encode_image(image_inputs)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    all_features.append(image_features.cpu())
                    frame_batch = []
            frame_idx += 1

        if frame_batch:
            image_inputs = torch.stack(
                [preprocess(img) for img in frame_batch]
            ).to(device)
            image_features = model.encode_image(image_inputs)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            all_features.append(image_features.cpu())

    cap.release()
    pbar.close()
    if not all_features:
        raise ValueError(f"No frames sampled for {video_path.name}")

    embeddings = torch.cat(all_features)
    mean_embedding = embeddings.mean(dim=0).numpy()
    return mean_embedding

In [6]:
print("Listing files from Google Drive...")
video_query = f"'{VIDEO_FOLDER_ID}' in parents and trashed=false"
video_files = drive.ListFile({'q': video_query}).GetList()

output_query = f"'{OUTPUT_FOLDER_ID}' in parents and trashed=false"
existing_embeddings = [f['title'] for f in drive.ListFile({'q': output_query}).GetList()]

print(f"Found {len(video_files)} videos in Drive.")
print(f"Found {len(existing_embeddings)} existing embeddings in output.")

for video_file in tqdm(video_files, desc="Processing Videos"):
    video_title = video_file['title']
    video_stem = Path(video_title).stem

    output_filename = f"{video_stem}_clip.npy" 

    if output_filename in existing_embeddings:
        continue

    temp_video_path = TEMP_FOLDER / video_title
    temp_npy_path = TEMP_FOLDER / output_filename

    try:
        print(f"Downloading {video_title}...")
        video_file.GetContentFile(str(temp_video_path))

        print(f"Processing {video_title}...")

        mean_embedding = extract_clip_embeddings(
            video_path=temp_video_path,
            model=model,
            preprocess=preprocess,
            device=device,
            frame_sample_rate=FRAME_SAMPLE_RATE,
            batch_size=BATCH_SIZE
        )
        np.save(temp_npy_path, mean_embedding)

        print(f"Uploading {output_filename}...")
        new_file = drive.CreateFile({
            'title': output_filename,
            'parents': [{'id': OUTPUT_FOLDER_ID}]
        })
        new_file.SetContentFile(str(temp_npy_path))
        new_file.Upload()

    except Exception as e:
        print(f"\n[ERROR] Failed to process {video_title}: {e}")

    finally:
        if temp_video_path.exists():
            os.remove(temp_video_path)

        if temp_npy_path.exists():
            try:
                os.remove(temp_npy_path)
            except PermissionError:
                print(f"[INFO] {temp_npy_path.name} is locked. Retrying in 5s...")
                try:
                    time.sleep(5)
                    os.remove(temp_npy_path)
                except Exception as e:
                    print(f"[WARNING] Could not delete temp file {temp_npy_path.name}: {e}")
            except Exception as e:
                 print(f"[WARNING] Could not delete temp file {temp_npy_path.name}: {e}")

print("\n--- Batch processing complete. ---")

Listing files from Google Drive...
Found 100 videos in Drive.
Found 100 existing embeddings in output.


Processing Videos:   0%|          | 0/100 [00:00<?, ?it/s]

Processing where_10.mp4...


Frames for where_10.mp4:   0%|          | 0/474 [00:00<?, ?it/s]

Uploading where_10_clip.npy...
[INFO] where_10_clip.npy is locked. Retrying in 5s...
[WARNING] Could not delete temp file where_10_clip.npy: [WinError 32] The process cannot access the file because it is being used by another process: 'where_10_clip.npy'
Processing where_9.mp4...


...

Uploading airball_3_clip.npy...
[INFO] airball_3_clip.npy is locked. Retrying in 5s...
[WARNING] Could not delete temp file airball_3_clip.npy: [WinError 32] The process cannot access the file because it is being used by another process: 'airball_3_clip.npy'
Processing airball_2.mp4...


Frames for airball_2.mp4:   0%|          | 0/401 [00:00<?, ?it/s]

Uploading airball_2_clip.npy...
[INFO] airball_2_clip.npy is locked. Retrying in 5s...
[WARNING] Could not delete temp file airball_2_clip.npy: [WinError 32] The process cannot access the file because it is being used by another process: 'airball_2_clip.npy'
Processing airball_1.mp4...


Frames for airball_1.mp4:   0%|          | 0/300 [00:00<?, ?it/s]

Uploading airball_1_clip.npy...
[INFO] airball_1_clip.npy is locked. Retrying in 5s...
[WARNING] Could not delete temp file airball_1_clip.npy: [WinError 32] The process cannot access the file because it is being used by another process: 'airball_1_clip.npy'

--- Batch processing complete. ---
